# OpEn Rust Usage Example 05

This example follows [Example 04](https://github.com/inmo-jang/optimisation_tutorial/blob/master/tools_examples/OpEn/examples_rust/OpEn_Rust_example_04.ipynb). 

In the previous example, we used the given cost function itself as an additional constraint. Here, we are just trying to solve the optimisation problem as it is. 


## Reminder 

- The ALM/PM solver of OpEn can address more general problems that involve constraints of the general form $F_1(u) \in C$ and $F_2(u) = 0$. 






For rust implementation in Jupyter notebook, we need to import "optimization_engine"
- In your local PC, it should be also declared in "Cargo.toml".
- In this jupyter notebook, we need to have "extern crate" as follows. 


In [2]:
extern crate optimization_engine;
use optimization_engine::{
    alm::*,
    constraints::*, panoc::*, *
};

## Problem Formulation

What we are going to solve is as follows: 

Minimise  $$f(u) = \frac{1}{2} ( u_1^2 + u_2^2) $$

subject to

$$2 u_1 -  u_3 + 1 = 0$$

### Objective Function

Suppose that
$$f(u) = \frac{1}{2} ( u_1^2 + u_2^2) $$

with gradient

$$\nabla f(u) = [ u_1 , u_2, 0 ] $$



In [3]:
pub fn f(u: &[f64], cost: &mut f64) -> Result<(), SolverError> {
    *cost = 0.5 * (u[0].powi(2)+u[1].powi(2));
    Ok(())
}


In [4]:
pub fn df(u: &[f64], grad: &mut [f64]) -> Result<(), SolverError> {
    grad[0] = u[0];
    grad[1] = u[1];
    grad[2] = 0.0;  
    Ok(())
}

### Constraints

We impose a constraint:

$$F_1(u) \in C$$
where 
$$F_1(u) = 2 u_1 -  u_3 + 1$$



$$C = \{0 \}$$



In [5]:
pub fn f1(u: &[f64], f1u: &mut [f64]) -> Result<(), SolverError> {
    f1u[0] = 2.0*u[0] - u[2] + 1.0;
    Ok(())
}

We need to define Jacobian product:

$$JF_1^{\top} \cdot d = 
\begin{bmatrix}
2 d_1\\
0 \\
-d_1
\end{bmatrix}
$$

where $J F_1$ is the Jacobian matrix of $F_1$ for given vectors $u \in \mathbb{R}^3$ (decision variables) and $d \in \mathbb{R}^1$ (dimension of constraints).

NOTE:

$$JF_1 = \begin{bmatrix}
\frac{\partial F_{1}}{\partial u_1} & \frac{\partial F_{1}}{\partial u_2} & \frac{\partial F_{1}}{\partial u_3} 
\end{bmatrix}
=
 \begin{bmatrix}
2 & 0 & -1
\end{bmatrix}
$$


In [6]:
pub fn f1_jacobian_product(u: &[f64], d: &[f64], res: &mut [f64]) -> Result<(), SolverError> {
    res[0] = 2.0*d[0];
    res[1] = 0.0;
    res[2] = -d[0];
    Ok(())
}

Now, the problem we need to solve is

$$\text{Minimise} f(u)$$
$$\text{subject to } -100 \le u_i \le 100 \text{  for all $i$}$$
$$F_1(u,p) \in C$$



### Invoking the main function

#### `AlmFactory` 
- construct $\psi$ and its gradient
- [Input structure](https://docs.rs/optimization_engine/0.6.2/src/optimization_engine/alm/alm_factory.rs.html#150-186) shoud be

```
pub fn new(
    f: Cost,
    df: CostGradient,
    mapping_f1: Option<MappingF1>,
    jacobian_mapping_f1_trans: Option<JacobianMappingF1Trans>,
    mapping_f2: Option<MappingF2>,
    jacobian_mapping_f2_trans: Option<JacobianMappingF2Trans>,
    set_c: Option<SetC>,
    n2: usize,
)

```

In [7]:
fn main() {
    let tolerance = 1e-5;
    let nx = 3; // problem_size: dimension of the decision variables
    let n1 = 1; // range dimensions of mappings F1
    let n2 = 0; // range dimensions of mappings F2
    let lbfgs_mem = 5; // memory of the LBFGS buffer
    
    // PANOCCache: All the information needed at every step of the algorithm
    let panoc_cache = PANOCCache::new(nx, tolerance, lbfgs_mem);
    
    // AlmCache: A cache structure that contains all the data 
    // that make up the state of the ALM/PM algorithm
    // (i.e., all those data that the algorithm updates)
    let mut alm_cache = AlmCache::new(panoc_cache, n1, n2);

    let set_c = Zero::new(); // Set C
    let bounds = Ball2::new(None, 100.0); // Set U
    let set_y = Ball2::new(None, 1e12);  // Set Y

    // AlmFactory: Prepare function psi and its gradient 
    // given the problem data such as f, del_f and 
    // optionally F_1, JF_1, C, F_2
    let factory = AlmFactory::new(
        f, // Cost function
        df, // Cost Gradient
        Some(f1), // MappingF1
        Some(f1_jacobian_product), // Jacobian Mapping F1 Trans
        NO_MAPPING, // MappingF2
        NO_JACOBIAN_MAPPING, // Jacobian Mapping F2 Trans
        Some(set_c), // Constraint set
        n2,
    );

    // Define an optimisation problem 
    // to be solved with AlmOptimizer
    let alm_problem = AlmProblem::new(
        bounds,
        Some(set_c),
        Some(set_y),
        |u: &[f64], xi: &[f64], cost: &mut f64| -> Result<(), SolverError> {
            factory.psi(u, xi, cost)
        },
        |u: &[f64], xi: &[f64], grad: &mut [f64]| -> Result<(), SolverError> {
            factory.d_psi(u, xi, grad)
        },
        Some(f1),
        NO_MAPPING,
        n1,
        n2,
    );

    let mut alm_optimizer = AlmOptimizer::new(&mut alm_cache, alm_problem)
        .with_delta_tolerance(1e-5)
        .with_max_outer_iterations(200)
        .with_epsilon_tolerance(1e-6)
        .with_initial_inner_tolerance(1e-2)
        .with_inner_tolerance_update_factor(0.5)
        .with_initial_penalty(100.0)
        .with_penalty_update_factor(1.05)
        .with_sufficient_decrease_coefficient(0.2)
        .with_initial_lagrange_multipliers(&vec![5.0; n1]);

    let mut u = vec![0.0; nx];
    let solver_result = alm_optimizer.solve(&mut u);
    let r = solver_result.unwrap();
    println!("\n\nSolver result : {:#.7?}\n", r);
    println!("Solution u = {:#.6?}", u);
}

Let's excute it.

In [8]:
main();




Solver result : AlmOptimizerStatus {
    exit_status: Converged,
    num_outer_iterations: 15,
    num_inner_iterations: 506,
    last_problem_norm_fpr: 0.0000008,
    lagrange_multipliers: Some(
        [
            0.0009983,
        ],
    ),
    solve_time: 209.8380000µs,
    penalty: 171.0339358,
    delta_y_norm: 0.0001620,
    f2_norm: 0.0000000,
}

Solution u = [
    -0.002094,
    0.000000,
    0.995811,
]


### This example is wrong

As you see above, the result is different from the previous example, where the solution was $u = [-0.44949, 0.0, 0.101021]$. 

**IMPORTANT:** It seems that we should have had the additional constraint as we did in Example 04. Let's do it like that again. 

Let's copy `f1` and `f1_jacobian_product` from the previous example. 


In [9]:
pub fn f1(u: &[f64], f1u: &mut [f64]) -> Result<(), SolverError> {
    f1u[0] = 2.0*u[0] - u[2] + 1.0;
    f1u[1] = 0.5 * (u[0].powi(2)+u[1].powi(2)) - u[2];
    Ok(())
}

pub fn f1_jacobian_product(u: &[f64], d: &[f64], res: &mut [f64]) -> Result<(), SolverError> {
    res[0] = 2.0*d[0] + u[0]*d[1];
    res[1] = u[1]*d[1];
    res[2] = -d[0] - d[1];
    Ok(())
}



As the dimension of F1 is changed, we also need to redefine `main()` as well.

In [11]:
fn main() {
    let tolerance = 1e-5;
    let nx = 3; // problem_size: dimension of the decision variables
    let n1 = 2; // range dimensions of mappings F1
    let n2 = 0; // range dimensions of mappings F2
    let lbfgs_mem = 5; // memory of the LBFGS buffer
    
    // PANOCCache: All the information needed at every step of the algorithm
    let panoc_cache = PANOCCache::new(nx, tolerance, lbfgs_mem);
    
    // AlmCache: A cache structure that contains all the data 
    // that make up the state of the ALM/PM algorithm
    // (i.e., all those data that the algorithm updates)
    let mut alm_cache = AlmCache::new(panoc_cache, n1, n2);

    let set_c = Zero::new(); // Set C
    let bounds = Ball2::new(None, 100.0); // Set U
    let set_y = Ball2::new(None, 1e12);  // Set Y

    // AlmFactory: Prepare function psi and its gradient 
    // given the problem data such as f, del_f and 
    // optionally F_1, JF_1, C, F_2
    let factory = AlmFactory::new(
        f, // Cost function
        df, // Cost Gradient
        Some(f1), // MappingF1
        Some(f1_jacobian_product), // Jacobian Mapping F1 Trans
        NO_MAPPING, // MappingF2
        NO_JACOBIAN_MAPPING, // Jacobian Mapping F2 Trans
        Some(set_c), // Constraint set
        n2,
    );

    // Define an optimisation problem 
    // to be solved with AlmOptimizer
    let alm_problem = AlmProblem::new(
        bounds,
        Some(set_c),
        Some(set_y),
        |u: &[f64], xi: &[f64], cost: &mut f64| -> Result<(), SolverError> {
            factory.psi(u, xi, cost)
        },
        |u: &[f64], xi: &[f64], grad: &mut [f64]| -> Result<(), SolverError> {
            factory.d_psi(u, xi, grad)
        },
        Some(f1),
        NO_MAPPING,
        n1,
        n2,
    );

    let mut alm_optimizer = AlmOptimizer::new(&mut alm_cache, alm_problem)
        .with_delta_tolerance(1e-5)
        .with_max_outer_iterations(200)
        .with_epsilon_tolerance(1e-6)
        .with_initial_inner_tolerance(1e-2)
        .with_inner_tolerance_update_factor(0.5)
        .with_initial_penalty(100.0)
        .with_penalty_update_factor(1.05)
        .with_sufficient_decrease_coefficient(0.2)
        .with_initial_lagrange_multipliers(&vec![5.0; n1]);

    let mut u = vec![0.0; nx];
    let solver_result = alm_optimizer.solve(&mut u);
    let r = solver_result.unwrap();
    println!("\n\nSolver result : {:#.7?}\n", r);
    println!("Solution u = {:#.6?}", u);
}


In [12]:
main();



Solver result : AlmOptimizerStatus {
    exit_status: Converged,
    num_outer_iterations: 15,
    num_inner_iterations: 80,
    last_problem_norm_fpr: 0.0000001,
    lagrange_multipliers: Some(
        [
            0.1834754,
            -0.1835201,
        ],
    ),
    solve_time: 163.7220000µs,
    penalty: 155.1328216,
    delta_y_norm: 0.0002070,
    f2_norm: 0.0000000,
}

Solution u = [
    -0.449490,
    0.000000,
    0.101021,
]


## Comparison with Example 04

- The outcome is the same.
- But, `solve_time` is 1.5 times higher (NB: it was 103 micro secs; and 50 of inner iterations). 